In [1]:
# 从sklearn 调入所需要的包
# from sklearn import datasets
from sklearn.model_selection import train_test_split #数据分隔出训练集和验证集
import lightgbm as lgb
import numpy as np
import pandas as pd
#导入精度和召回
from sklearn.metrics import precision_score, recall_score
import lightgbm as lgb

In [11]:
path='./'
train = pd.read_csv(path+'input/train_set.csv')
test = pd.read_csv(path+'input/test_set.csv')
# print(train.info())
# print(test.info())

In [8]:
# train.describe()

In [9]:
# test.describe()

In [10]:
# train.corr()

In [12]:
train.job.value_counts()

blue-collar      5456
management       5296
technician       4241
admin.           2909
services         2342
retired          1273
self-employed     884
entrepreneur      856
unemployed        701
housemaid         663
student           533
unknown           163
Name: job, dtype: int64

In [18]:
test['y']=-1
print(len(test.columns))

18


In [19]:
data = train.append(test).reset_index(drop=True)
# data = pd.concat([train,test]).rese_index(drop=True)
print(data.head())

   ID  age         job   marital  education default  balance housing loan  \
0   1   43  management   married   tertiary      no      291     yes   no   
1   2   42  technician  divorced    primary      no     5076     yes   no   
2   3   47      admin.   married  secondary      no      104     yes  yes   
3   4   28  management    single  secondary      no     -994     yes  yes   
4   5   42  technician  divorced  secondary      no     2974     yes   no   

    contact  day month  duration  campaign  pdays  previous poutcome  y  
0   unknown    9   may       150         2     -1         0  unknown  0  
1  cellular    7   apr        99         1    251         2    other  0  
2  cellular   14   jul        77         2     -1         0  unknown  0  
3  cellular   18   jul       174         2     -1         0  unknown  0  
4   unknown   21   may       187         5     -1         0  unknown  0  


In [20]:
path='./'
import os
import json
import gc
# os.system('pip install numba')
from numba import jit
#tqdm
# os.system('pip install tqdm')
from tqdm import tqdm_notebook
from tqdm import tqdm

#Integrated model
# os.system('pip install lightgbm')
import lightgbm as lgb
# os.system('pip install catboost==0.15.2')
import catboost as cbt
# os.system('pip install xgboost')
# import xgboost as xgb

#base import 
import numpy as np
import pandas as pd

# about sklearn
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold, KFold, RepeatedKFold
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler as std
from sklearn.kernel_ridge import KernelRidge
from sklearn.metrics import f1_score
#about time
import time
import datetime 
from datetime import datetime, timedelta

#Garbage collection
import gc
# scipy
from scipy.signal import hilbert
from scipy.signal import hann
from scipy.signal import convolve
from scipy import stats
import scipy.spatial.distance as dist
#other
from collections import Counter 
from statistics import mode 
    #warning
import warnings
warnings.filterwarnings("ignore")
import json 
import math
from itertools import product
import ast 

In [35]:
pd.set_option('display.max_columns', 1000)

In [27]:
# help(data.select_dtypes)
cat_col = [i for i in data.select_dtypes(object).columns if i not in ['ID','y']]
num_features = [i for i in data.select_dtypes('number').columns if i not in ['ID','y']]

In [33]:
# count
for i in tqdm_notebook(cat_col):
    lbl = LabelEncoder()
    # groupby基本操作
    data['count_' + i] = data.groupby([i])[i].transform('count')
    data[i] = lbl.fit_transform(data[i].astype(str))

1ID Int客户唯一标识

2age Int客户年龄

3job String客户的职业

4marital String婚姻状况

5education String受教育水平

6default String是否有违约记录

7balance Int每年账户的平均余额

8housing String是否有住房贷款

9loan String是否有个人贷款

10contact String与客户联系的沟通方式

11day Int最后一次联系的时间（几号）

12month String最后一次联系的时间（月份）

13duration Int最后一次联系的交流时长

14campaign Int在本次活动中，与该客户交流过的次数

15pdays Int距离上次活动最后一次联系该客户，过去了多久（999表示没有联系过）16previousInt在本次活动之前，与该客户交流过的次数

17poutcome String上一次活动的结果

18y Int预测客户是否会订购定期存款业务

In [36]:
data.head()

,ID,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y,count_job,count_marital,count_education,count_default,count_housing,count_loan,count_contact,count_month,count_poutcome
0,1,43,4,1,2,0,291,1,0,2,9,8,150,2,-1,0,3,0,7595,21857,10654,35524,19959,30363,10393,10957,29554
1,2,42,9,0,0,0,5076,1,0,0,7,0,99,1,251,2,1,0,6083,4140,5493,35524,19959,30363,23437,2365,1475
2,3,47,0,1,1,0,104,1,1,0,14,5,77,2,-1,0,3,0,4124,21857,18509,35524,19959,5806,23437,5569,29554
3,4,28,4,2,1,0,-994,1,1,0,18,5,174,2,-1,0,3,0,7595,10172,18509,35524,19959,5806,23437,5569,29554
4,5,42,9,0,1,0,2974,1,0,2,21,8,187,5,-1,0,3,0,6083,4140,18509,35524,19959,30363,10393,10957,29554


In [37]:
feats = [i for i in data.columns if i not in ['ID','y']]
print(feats)

['age', 'job', 'marital', 'education', 'default', 'balance', 'housing', 'loan', 'contact', 'day', 'month', 'duration', 'campaign', 'pdays', 'previous', 'poutcome', 'count_job', 'count_marital', 'count_education', 'count_default', 'count_housing', 'count_loan', 'count_contact', 'count_month', 'count_poutcome']


In [38]:
model = lgb.LGBMClassifier(
        boosting_type="gbdt", num_leaves=30, reg_alpha=0, reg_lambda=0.,
    max_depth=-1, n_estimators=1500, objective='binary',metric= 'auc',
    subsample=0.95, colsample_bytree=0.7, subsample_freq=1,
    learning_rate=0.02, random_state=2017
    )
train_x =data[data['y']!=-1][feats]
train_y =data[data['y']!=-1]['y']
testx= data[data['y']==-1][feats]
# train_x, test_x, train_y, test_y = train_test_split(data1, label1, test_size=0.3, random_state=42)
model.fit(train_x,train_y)
test_pre = model.predict_proba(testx)[:,1]#, num_iteration=model.best_iteration

In [39]:
print(len(test_pre))

10852


In [40]:
pre=data[data['y']==-1][['ID']]
pre['pred']=test_pre
print(len(pre))

10852


In [41]:
pre.head()

,ID,pred
25317,25318,0.034399
25318,25319,0.005326
25319,25320,0.001620
25320,25321,0.771375
25321,25322,0.018045


In [42]:
pre.to_csv('baseline2.csv',index=False )

In [44]:
#5折交叉验证
from sklearn.model_selection import KFold
n_splits=10
kfold = KFold(n_splits=10, shuffle=True, random_state=42)
train_x = data[data['y']!=-1][feats]
train_y = data[data['y']!=-1]['y']
res=data[data['y']==-1][['ID']]

test_x= data[data['y']==-1][feats]
res['pred'] = 0
for train_idx, val_idx in kfold.split(train_x):
    model.random_state = model.random_state + 1
    train_x1 = train_x.loc[train_idx]
    train_y1 = train_y.loc[train_idx]
    test_x1 = train_x.loc[val_idx]
    test_y1 = train_y.loc[val_idx]
    #,(vali_x,vali_y)
    model.fit(train_x1, train_y1,eval_set=[(train_x1, train_y1),(test_x1, test_y1)],eval_metric='auc',early_stopping_rounds=100)
    res['pred'] += model.predict_proba(test_x)[:,1]

res['pred'] = res['pred']/10
res.to_csv('./submission10_fold.csv', index=False)

[1]	training's auc: 0.763373	valid_1's auc: 0.76062
Training until validation scores don't improve for 100 rounds.
[2]	training's auc: 0.908515	valid_1's auc: 0.895653
[3]	training's auc: 0.909321	valid_1's auc: 0.898645
[4]	training's auc: 0.903865	valid_1's auc: 0.892363
[5]	training's auc: 0.916257	valid_1's auc: 0.903828
[6]	training's auc: 0.924793	valid_1's auc: 0.914172
[7]	training's auc: 0.926916	valid_1's auc: 0.916287
[8]	training's auc: 0.928724	valid_1's auc: 0.919389
[9]	training's auc: 0.928561	valid_1's auc: 0.918357
[10]	training's auc: 0.927633	valid_1's auc: 0.917046
[11]	training's auc: 0.929848	valid_1's auc: 0.918533
[12]	training's auc: 0.9314	valid_1's auc: 0.921337
[13]	training's auc: 0.932031	valid_1's auc: 0.922106
[14]	training's auc: 0.931416	valid_1's auc: 0.92091
[15]	training's auc: 0.931219	valid_1's auc: 0.920494
[16]	training's auc: 0.930608	valid_1's auc: 0.918564
[17]	training's auc: 0.931586	valid_1's auc: 0.920206
[18]	training's auc: 0.93233	val

[151]	training's auc: 0.95027	valid_1's auc: 0.93045
[152]	training's auc: 0.950356	valid_1's auc: 0.930442
[153]	training's auc: 0.950398	valid_1's auc: 0.930391
[154]	training's auc: 0.950494	valid_1's auc: 0.930609
[155]	training's auc: 0.950577	valid_1's auc: 0.930613
[156]	training's auc: 0.950687	valid_1's auc: 0.930548
[157]	training's auc: 0.950768	valid_1's auc: 0.930578
[158]	training's auc: 0.95085	valid_1's auc: 0.930585
[159]	training's auc: 0.95096	valid_1's auc: 0.930628
[160]	training's auc: 0.951028	valid_1's auc: 0.930676
[161]	training's auc: 0.951069	valid_1's auc: 0.930725
[162]	training's auc: 0.951144	valid_1's auc: 0.930722
[163]	training's auc: 0.951237	valid_1's auc: 0.930685
[164]	training's auc: 0.951335	valid_1's auc: 0.930843
[165]	training's auc: 0.951465	valid_1's auc: 0.930878
[166]	training's auc: 0.951631	valid_1's auc: 0.9311
[167]	training's auc: 0.951685	valid_1's auc: 0.931106
[168]	training's auc: 0.951764	valid_1's auc: 0.931109
[169]	training's

[300]	training's auc: 0.961094	valid_1's auc: 0.932919
[301]	training's auc: 0.961155	valid_1's auc: 0.932886
[302]	training's auc: 0.961221	valid_1's auc: 0.932964
[303]	training's auc: 0.961288	valid_1's auc: 0.932986
[304]	training's auc: 0.961346	valid_1's auc: 0.933019
[305]	training's auc: 0.961424	valid_1's auc: 0.933017
[306]	training's auc: 0.961481	valid_1's auc: 0.933
[307]	training's auc: 0.961542	valid_1's auc: 0.932998
[308]	training's auc: 0.961604	valid_1's auc: 0.932998
[309]	training's auc: 0.961671	valid_1's auc: 0.933028
[310]	training's auc: 0.961766	valid_1's auc: 0.933103
[311]	training's auc: 0.961819	valid_1's auc: 0.933098
[312]	training's auc: 0.96188	valid_1's auc: 0.933081
[313]	training's auc: 0.961943	valid_1's auc: 0.933051
[314]	training's auc: 0.962007	valid_1's auc: 0.933054
[315]	training's auc: 0.962071	valid_1's auc: 0.933006
[316]	training's auc: 0.962125	valid_1's auc: 0.933013
[317]	training's auc: 0.962176	valid_1's auc: 0.933126
[318]	training

[449]	training's auc: 0.969565	valid_1's auc: 0.934013
[450]	training's auc: 0.969611	valid_1's auc: 0.934001
[451]	training's auc: 0.969663	valid_1's auc: 0.934038
[452]	training's auc: 0.969721	valid_1's auc: 0.934042
[453]	training's auc: 0.96978	valid_1's auc: 0.934046
[454]	training's auc: 0.969831	valid_1's auc: 0.934058
[455]	training's auc: 0.969894	valid_1's auc: 0.934071
[456]	training's auc: 0.969948	valid_1's auc: 0.934039
[457]	training's auc: 0.970008	valid_1's auc: 0.93408
[458]	training's auc: 0.970057	valid_1's auc: 0.934061
[459]	training's auc: 0.970093	valid_1's auc: 0.934073
[460]	training's auc: 0.970137	valid_1's auc: 0.934086
[461]	training's auc: 0.97019	valid_1's auc: 0.934145
[462]	training's auc: 0.970241	valid_1's auc: 0.934158
[463]	training's auc: 0.970273	valid_1's auc: 0.934148
[464]	training's auc: 0.970321	valid_1's auc: 0.934135
[465]	training's auc: 0.970357	valid_1's auc: 0.934167
[466]	training's auc: 0.970429	valid_1's auc: 0.93421
[467]	training

[598]	training's auc: 0.975908	valid_1's auc: 0.933887
[599]	training's auc: 0.975962	valid_1's auc: 0.933877
[600]	training's auc: 0.976009	valid_1's auc: 0.933865
[601]	training's auc: 0.976052	valid_1's auc: 0.933905
[602]	training's auc: 0.976085	valid_1's auc: 0.933904
[603]	training's auc: 0.976138	valid_1's auc: 0.933954
[604]	training's auc: 0.976176	valid_1's auc: 0.933961
[605]	training's auc: 0.976196	valid_1's auc: 0.933952
[606]	training's auc: 0.976231	valid_1's auc: 0.933943
[607]	training's auc: 0.976273	valid_1's auc: 0.934008
[608]	training's auc: 0.976305	valid_1's auc: 0.934033
[609]	training's auc: 0.976349	valid_1's auc: 0.934054
[610]	training's auc: 0.976395	valid_1's auc: 0.934048
[611]	training's auc: 0.976431	valid_1's auc: 0.934042
[612]	training's auc: 0.976469	valid_1's auc: 0.934018
[613]	training's auc: 0.976501	valid_1's auc: 0.934038
[614]	training's auc: 0.97655	valid_1's auc: 0.934013
[615]	training's auc: 0.976591	valid_1's auc: 0.933967
[616]	train

...

[789]	training's auc: 0.981946	valid_1's auc: 0.939239
[790]	training's auc: 0.981985	valid_1's auc: 0.939223
[791]	training's auc: 0.982005	valid_1's auc: 0.93922
[792]	training's auc: 0.982031	valid_1's auc: 0.939212
[793]	training's auc: 0.982058	valid_1's auc: 0.939195
[794]	training's auc: 0.982093	valid_1's auc: 0.939163
[795]	training's auc: 0.982135	valid_1's auc: 0.939188
[796]	training's auc: 0.982169	valid_1's auc: 0.939201
[797]	training's auc: 0.982201	valid_1's auc: 0.939256
[798]	training's auc: 0.982234	valid_1's auc: 0.939261
[799]	training's auc: 0.982251	valid_1's auc: 0.939251
[800]	training's auc: 0.98229	valid_1's auc: 0.939226
[801]	training's auc: 0.982308	valid_1's auc: 0.939229
[802]	training's auc: 0.982322	valid_1's auc: 0.939265
[803]	training's auc: 0.982338	valid_1's auc: 0.939237
[804]	training's auc: 0.982363	valid_1's auc: 0.939244
[805]	training's auc: 0.982409	valid_1's auc: 0.939247
[806]	training's auc: 0.982433	valid_1's auc: 0.939256
[807]	traini

[15]	training's auc: 0.932907	valid_1's auc: 0.911951
[16]	training's auc: 0.933346	valid_1's auc: 0.912351
[17]	training's auc: 0.933596	valid_1's auc: 0.912861
[18]	training's auc: 0.933958	valid_1's auc: 0.912622
[19]	training's auc: 0.934191	valid_1's auc: 0.912321
[20]	training's auc: 0.934153	valid_1's auc: 0.91213
[21]	training's auc: 0.934333	valid_1's auc: 0.911913
[22]	training's auc: 0.934524	valid_1's auc: 0.912377
[23]	training's auc: 0.934412	valid_1's auc: 0.912512
[24]	training's auc: 0.934553	valid_1's auc: 0.912317
[25]	training's auc: 0.934829	valid_1's auc: 0.912522
[26]	training's auc: 0.93508	valid_1's auc: 0.912959
[27]	training's auc: 0.935162	valid_1's auc: 0.913208
[28]	training's auc: 0.935281	valid_1's auc: 0.913267
[29]	training's auc: 0.935382	valid_1's auc: 0.913414
[30]	training's auc: 0.935737	valid_1's auc: 0.914188
[31]	training's auc: 0.935912	valid_1's auc: 0.91446
[32]	training's auc: 0.93597	valid_1's auc: 0.914299
[33]	training's auc: 0.936013	va

[166]	training's auc: 0.952399	valid_1's auc: 0.922072
[167]	training's auc: 0.952463	valid_1's auc: 0.922167
[168]	training's auc: 0.952603	valid_1's auc: 0.922209
[169]	training's auc: 0.952666	valid_1's auc: 0.922343
[170]	training's auc: 0.95272	valid_1's auc: 0.922385
[171]	training's auc: 0.952786	valid_1's auc: 0.922441
[172]	training's auc: 0.952849	valid_1's auc: 0.922348
[173]	training's auc: 0.95291	valid_1's auc: 0.922387
[174]	training's auc: 0.95294	valid_1's auc: 0.922382
[175]	training's auc: 0.953008	valid_1's auc: 0.922418
[176]	training's auc: 0.95306	valid_1's auc: 0.922519
[177]	training's auc: 0.953137	valid_1's auc: 0.922545
[178]	training's auc: 0.953186	valid_1's auc: 0.922483
[179]	training's auc: 0.953212	valid_1's auc: 0.922486
[180]	training's auc: 0.953275	valid_1's auc: 0.922414
[181]	training's auc: 0.953315	valid_1's auc: 0.922362
[182]	training's auc: 0.953395	valid_1's auc: 0.922357
[183]	training's auc: 0.953463	valid_1's auc: 0.92243
[184]	training'

[315]	training's auc: 0.962887	valid_1's auc: 0.923846
[316]	training's auc: 0.962929	valid_1's auc: 0.923842
[317]	training's auc: 0.96299	valid_1's auc: 0.923833
[318]	training's auc: 0.963084	valid_1's auc: 0.923968
[319]	training's auc: 0.963155	valid_1's auc: 0.923992
[320]	training's auc: 0.963248	valid_1's auc: 0.924003
[321]	training's auc: 0.963304	valid_1's auc: 0.924001
[322]	training's auc: 0.963367	valid_1's auc: 0.923969
[323]	training's auc: 0.963436	valid_1's auc: 0.923955
[324]	training's auc: 0.963483	valid_1's auc: 0.92394
[325]	training's auc: 0.963526	valid_1's auc: 0.923982
[326]	training's auc: 0.96359	valid_1's auc: 0.924027
[327]	training's auc: 0.963647	valid_1's auc: 0.924015
[328]	training's auc: 0.963711	valid_1's auc: 0.924014
[329]	training's auc: 0.963778	valid_1's auc: 0.923976
[330]	training's auc: 0.963845	valid_1's auc: 0.924014
[331]	training's auc: 0.9639	valid_1's auc: 0.923987
[332]	training's auc: 0.963957	valid_1's auc: 0.924008
[333]	training'

[464]	training's auc: 0.97071	valid_1's auc: 0.924017
[465]	training's auc: 0.970748	valid_1's auc: 0.923989
[466]	training's auc: 0.970797	valid_1's auc: 0.924001
[467]	training's auc: 0.970837	valid_1's auc: 0.924024
[468]	training's auc: 0.970876	valid_1's auc: 0.924
[469]	training's auc: 0.97091	valid_1's auc: 0.923982
[470]	training's auc: 0.970958	valid_1's auc: 0.924015
[471]	training's auc: 0.971	valid_1's auc: 0.924008
[472]	training's auc: 0.97104	valid_1's auc: 0.924032
[473]	training's auc: 0.971075	valid_1's auc: 0.92401
[474]	training's auc: 0.971134	valid_1's auc: 0.923994
[475]	training's auc: 0.971164	valid_1's auc: 0.924036
[476]	training's auc: 0.971193	valid_1's auc: 0.924007
[477]	training's auc: 0.971229	valid_1's auc: 0.923976
[478]	training's auc: 0.971271	valid_1's auc: 0.924006
[479]	training's auc: 0.971326	valid_1's auc: 0.923973
[480]	training's auc: 0.971365	valid_1's auc: 0.923924
[481]	training's auc: 0.971399	valid_1's auc: 0.923978
[482]	training's auc